In [2]:
cd 'OneDrive - Intel Corporation\Documents\research_outside\geometricLWE\sage_notebook'

/home/sage/OneDrive - Intel Corporation/Documents/research_outside/geometricLWE/sage_notebook


In [2]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
demo = False
from scipy.io import loadmat
import random

In [56]:
load("../framework/geometry.sage")
load("../framework/instance_gen.sage")


"""  Example
Uncomment the following to get an example
of the detailed computation (without redundancy)
"""
upper_bound_as_prob = True
to_print = True
# demo = True
# logging("--- Demonstration mode (no redundancy of the attacks) ---")

for params in ['CCS2']:#[ 'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:#['NIST1', 'NIST2']:#[ 'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:#[ 'CCS1', 'CCS2']:#['CCS3', 'CCS4', 'NIST1', 'NIST2']:#['CCS2']: # [ 'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:
    logging("Set of parameters: " + params)

    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
        n = 352
        m = 352
        q = 2 ** 11
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3


    """  Original Security   """

    A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                               n,
                                               q, m, D_s, D_s, verbosity=0)
    dbdd.integrate_q_vectors(q, indices=range(0, m))
    (beta, _) = dbdd.estimate_attack()
    logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

    """  Refined Side channel attack  """

    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]


    if demo:
        A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                  n,
                                                  q, m, D_s, D_s, verbosity=2)
        measured = [simu_measured(dbdd.u[0, m + i], measures) for i in range(n)]
        if upper_bound_as_prob:
            estimate_SCA_prob(50, dbdd, measured, 0.5, D_s, D_s)
        else:
            estimate_SCA_num(50, dbdd, measured, 175, D_s, D_s)
    else:
        """  Averaging
        The following averages the measures to get accurate data
        for the paper. The averaging mode is quite long.
        """
        nb_tests_per_params = 1

        # Chosen values for the number of guesses
        if upper_bound_as_prob:
            if params == 'CCS1':
                max_guesses = 0.87
            elif params == 'CCS2':
                max_guesses = 0.77
            elif params == 'CCS3':
                max_guesses = 0.81
            elif params == 'CCS4':
                max_guesses = 0.84
            elif params == 'NIST1':
                max_guesses = 0.86
            elif params == 'NIST2':
                max_guesses = 0.64
        else:
            if params == 'CCS1':
                max_guesses = 175 + 100
            elif params == 'CCS2':
                max_guesses = 300
            elif params == 'CCS3':
                max_guesses = 250 + 125
            elif params == 'CCS4':
                max_guesses = 250 + 125
            elif params == 'NIST1':
                max_guesses = 100 + 75
            elif params == 'NIST2':
                max_guesses = 250 + 125

        beta = 0
        beta_hybrid = 0
        proba_success = 0
        guesses_average = 0
        beta_hybrid_before_q = 0
        for i in range(nb_tests_per_params):
            print("SAMPLE = ", i)
            A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                      n,
                                                      q, m, D_s, D_s, verbosity=0)
            measured = [simu_measured(dbdd.u[0, m + i], measures) for i in range(n)]
            if upper_bound_as_prob:
                b, b_hybrid, p_success, guesses, beta_before_q, testing_data, testing_data2= estimate_SCA_prob(None, dbdd,
                                                  measured, max_guesses, D_s, to_print)
            else:
                b, b_hybrid, p_success, guesses = estimate_SCA_num(None, dbdd,
                                                  measured, max_guesses, D_s)
            beta += b
            beta_hybrid += b_hybrid
            proba_success += p_success
            guesses_average += guesses
            beta_hybrid_before_q += beta_before_q

        beta /= nb_tests_per_params
        beta_hybrid /= nb_tests_per_params
        proba_success /= nb_tests_per_params
        guesses_average /= nb_tests_per_params
        beta_hybrid_before_q /= nb_tests_per_params
        
        logging("Attack with hints: %3.2f bikz" % beta, style="HEADER")
        logging("Attack with hints and guess before q_vectors: %3.2f bikz"% beta_hybrid_before_q, style="HEADER")
        logging("Attack with hints and guess: %3.2f bikz"% beta_hybrid, style="HEADER")
        logging("Number of guesses: %4d" % guesses_average, style="HEADER")
        logging("Success probability: %3.2f" %proba_success, style="HEADER")


 Set of parameters: CCS2 
 Attack without hints:  447.85 bikz 
SAMPLE =  0
proba_best = 1
      Another Real Perfect Hint /////     
smallest and second smallest var from SCA: [1.0256552551712872e-16, 1.2290515827295678e-15]
number of elements in intersect_indices 225
map_nonzero_to_full {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60, 60: 61, 61: 62, 62: 63, 63: 64, 64: 65, 65: 66, 66: 67, 67: 68, 68: 69, 69: 70, 70: 71, 71: 72, 72: 73, 73: 74, 74: 75, 75: 76, 76: 77, 77: 78, 78: 79, 79: 80, 80: 81, 81: 82, 82: 83, 83: 84, 84: 85, 85: 86, 86: 87, 87: 88, 88: 89, 89: 90, 90: 91, 9

/opt/sagemath-9.3/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:67: RuntimeWarning: divide by zero encountered in log


poster_beta 2
right after poster: (# of guesses, last prob) =  (388, 0.9880521285499416)
new S dimention after intersection:  1185
intersect_beta before q:  2
coordinate 592
      Real Perfect Hint in Hybrid Phase     
count_guess1_better! 0.8105125114745027
Comparison (best2, best1, u):  -1 -3 -3
count_both_not_good! 0.8593516839459061 0.795999880725785
Comparison (best2, best1, u):  1 1 2
count_both_not_good! 0.997754916329426 0.9450014458926209
Comparison (best2, best1, u):  1 1 2
count_guess1_better! 0.8330575771284293
Comparison (best2, best1, u):  -2 -3 -3
count_both_not_good! 0.7471465507580474 0.5433130201135727
Comparison (best2, best1, u):  1 1 2
count_guess1_better! 0.7379712755279101
Comparison (best2, best1, u):  1 2 2
count_guess2_better! 0.6975854756666959
Comparison (best2, best1, u):  1 2 1
count_guess2_better! 0.7137827828603174
Comparison (best2, best1, u):  1 2 1
count_both_not_good! 0.8154459678517224 0.7368033039342674
Comparison (best2, best1, u):  1 1 2
count_gu

In [56]:
[set_guess_0, set_guess2_better_g, set_guess1_better_g, set_both_not_good_g, set_guess_g, set_guess2_better_1, set_guess1_better_1, set_both_not_good_1, set_guess_1]

2

In [69]:
testing_data2[3]

{898: 0.9996671492274908,
 1083: 0.999279276958022,
 885: 0.9953862345423252,
 692: 0.992783632131633}

In [73]:
for i in testing_data2[3].keys():
    print(i)
    print(i in testing_data2[-1].keys())

898
True
1083
True
885
True
692
True


In [79]:
set_guess_g = testing_data2[4]
set_guess_1 = testing_data2[-1]

In [83]:
ctr = 0
for i in set_guess_1.keys():
    if not i in set_guess_g.keys():
        ctr += 1
        print(i, set_guess_1[i])
print(ctr)

817 1.0
630 0.9999999999999999
1112 0.9999999999999998
622 0.9999999999999987
930 0.9999999999999972
1075 0.9999999999999972
1120 0.999999999999995
754 0.9999999999999926
685 0.9999999999999885
804 0.9999999999999634
723 0.9999999999999536
786 0.9999999999999497
1035 0.9999999999999395
694 0.9999999999999355
856 0.9999999999998942
893 0.9999999999998934
886 0.9999999999997439
814 0.9999999999996824
1155 0.9999999999996698
647 0.9999999999996572
982 0.9999999999995597
888 0.9999999999995094
609 0.9999999999994972
947 0.9999999999994835
963 0.9873139700027388
602 0.9867627722882205
765 0.984766386007224
606 0.983710114996377
28


In [74]:
proba_best = testing_data[0]

In [76]:
for i in testing_data2[3].keys():
    print(i, proba_best[i-m])

898 0.9587837582543113
1083 0.9542963440404605
885 0.9380247042868486
692 0.9321680904040357


In [55]:
def estimate_SCA_prob(report_every, dbdd, measured, max_guesses, D_s, to_print):
    #Upper bounding the maximum number of guessed coordinate
    """ 
    This function evaluates the security loss after Bos et al attack
    :report_every: an integer that give the frequency of
    logging (None for no logging)
    :dbdd: instance of the class DBDD
    :measured: table representing the (simulated) information
    given by Bos et al attack
    :max_guesses: integer for upperbounding the number of guesses
    """
    add_poster = True
    MINIMUM_VAR = 0.01
    proba_best = {}
    Id = identity_matrix(n + m)
    list_av = [0] * n
    list_var = [0] * n
    _, s_s = average_variance(D_s)
    intersect_indices = []
    coord_best1 = {} ###########################guess test
    # sca_var_zero_indices = []
    for i in range(n):
        apost_dist = measurement_to_aposteriori(measured[i])
        if apost_dist is not None:
            proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
            dist_j = [(j, apost_dist[j]) for j in apost_dist.keys()] ######################################### guess test
            coord_best1[i] = max(dist_j, key=lambda x:x[1])[0]  ######################################### guess test
            av, var = average_variance(apost_dist)
            av = float(av)
            var = float(var)
            list_av[i] = av
            list_var[i] = var
            if var == 0.0:
                print("yes, var == 0")
                # sca_var_zero_indices += [i]
            if av*av > s_s - var and var != 0.0:
                intersect_indices += [i]
        else:
            print("when apost_dist in this coordinate is None")
            av = None
            var = None
        v = vec(Id[i])
        if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
            verbose = 2 
        else:
            verbose = 0
        dbdd.verbosity = verbose
        if verbose == 2:
            logging("[...%d]" % report_every, newline=False)
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
        if var == 0:
            print("var == 0")
            logging("     Real Perfect Hint /////    ", style="HEADER") #######TO FIX SINGULAR MATRIX WOULD OCCUR
        if proba_best[i] == 1:
            print("proba_best = 1")
            logging("     Another Real Perfect Hint /////    ", style="HEADER")
            # list_var[i] = MINIMUM_VAR
    #         intersect_indices.remove(i)
    #         print("remove indices")
    # print(intersect_indices)
    print("smallest and second smallest var from SCA:", sorted(list_var)[:2])
    if to_print:    
        print("number of elements in intersect_indices", len(intersect_indices))

    # if to_print:
    #     print("--comparing real secret v.s. SCA distribution--")
    #     for i in range(len(list_av)):
    #         print(list_av[i], list_var[i], dbdd.u[0, m+i])
    #     print("--finish comparing--")
    #####Intersection Phase####
#     dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices, to_print)
    set_guess_0 = {}
    if not add_poster:
        dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices, to_print)
        dbdd.mu = intersect_mu

    else:
        # intersect_S, intersect_mu, poster_S, poster_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices, to_print)
        #testing:
        for i in range(n):
            if i not in intersect_indices:
                list_var[0] = 0.0
                break
        
        intersect_S, intersect_mu, poster_S, poster_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices)
        #integrating SCA through posteriori distribution
        dbdd.S = poster_S
        dbdd.mu = poster_mu
        
        if to_print:
            (poster_beta, _) = dbdd.estimate_attack()
            print("poster_beta", poster_beta)
            # dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(0, n)) #n, n + m
            # (poster_beta_after_q, _) = dbdd.estimate_attack()
            # print("poster_beta_after_q: ", poster_beta_after_q)
        
        sorted_guesses0 = sorted(proba_best.items(),
                            key=lambda kv: - kv[1]) #sorted index

        sorted_guesses0 = [sorted_guesses0[i][0] for i in range(len(sorted_guesses0))]

        proba_success0 = 1.
        guesses0 = 0

        last_included_prob0 = 0 ################
        # set_guess_0 = {} ##################################
        for t_semi in sorted_guesses0: #############
            t = t_semi + m ##############################
            # print("(guesses, max_guesses)=", guesses, max_guesses)
            if proba_success0*proba_best[t-m] >= 0.77:#proba_success*proba_best2[t] >= 0.77: #(guesses <= max_guesses):(guesses <= max_guesses): proba_success*proba_best2[t] >= max_guesses  #################################
                # print("comparing prob_best in guess phase: ", proba_best[t-n], proba_best2[t])
                v = vec(Id[t])
                #perfect hint deleted
                guesses0 += 1
                proba_success0 *= proba_best[t-m] #proba_best2[t]  #####################################################
                last_included_prob0 = proba_best[t-m] #proba_best2[t]  ############################################################
                set_guess_0[t] = proba_best[t-m]
        print("right after poster: (# of guesses, last prob) = ", (guesses0, last_included_prob0))


        dbdd.S = intersect_S
        dbdd.mu = intersect_mu
        print("new S dimention after intersection: ",len(list(intersect_S)))
        # if to_print:
        #     print("after intersection, dbdd.u: ", RR(dbdd.mu)[0, -5:])
    

    # if to_print:
    #     print("ellip_norm after intersection: ")

    #     # print((dbdd.u - dbdd.mu).nrows(), (dbdd.u - dbdd.mu).ncols())
    #     # print((matrix(np_inv(diagonal_matrix(dbdd.S)))).nrows(), (matrix(np_inv(diagonal_matrix(dbdd.S)))).ncols())  
    #     print(scal((dbdd.u - dbdd.mu) * matrix(np_inv(diagonal_matrix(dbdd.S)))/(n + m + 1) * (dbdd.u - dbdd.mu).T)) #should be <=1
    (beta_inter_before_q, _) = dbdd.estimate_attack()
    print("intersect_beta before q: ", beta_inter_before_q)


    
    
    
    if report_every is not None:
        logging("     Hybrid attack estimation     ", style="HEADER")
    
    intersect_var = list(dbdd.S)
    intersect_av = list(dbdd.mu[0])
    proba_best2 = {}
    coord_best2 = {}
    # print("len(intersect_var)", len(intersect_var))
    count_bad = 0
    count_guess2_better = 0 ###############
    count_guess1_better = 0 ###############
    count_both_not_good = 0 ################
    count_total_guess12 = 0 ################
    for k in range(len(intersect_var) - 1): #####double check 5/16
        var = intersect_var[k]
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
            continue
        if var == 0:# + 0.02:
            print("coordinate", k)            
            v = vec(Id[k])

            logging("     Real Perfect Hint in Hybrid Phase    ", style="HEADER")
            # intersect_var[k] = MINIMUM_VAR
            # dbdd.S[k] = MINIMUM_VAR
            dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        # print("k = ", k,intersect_var[k])

        if k - m in intersect_indices:
            if var == 0:
                print("var == 0 in after intersection!")
            proba_best2[k], coord_best2[k] = var_to_prob(intersect_var[k], intersect_av[k], D_s)  #######################guess test
            # print("Comparison (best2, best1, u): ", coord_best2[k], coord_best1[k-m], dbdd.u[0, k]) 
            count_total_guess12 += 1  
            if coord_best2[k] != dbdd.u[0, k] and coord_best1[k-m] == dbdd.u[0, k]:
                print("count_guess1_better!", proba_best[k-m])
                count_guess1_better += 1
                print("Comparison (best2, best1, u): ", coord_best2[k], coord_best1[k-m], dbdd.u[0, k]) 
            if coord_best2[k] == dbdd.u[0, k] and coord_best1[k-m] != dbdd.u[0, k]:
                print("count_guess2_better!", proba_best2[k])
                count_guess2_better += 1
                print("Comparison (best2, best1, u): ", coord_best2[k], coord_best1[k-m], dbdd.u[0, k]) 
            if coord_best2[k] != dbdd.u[0, k] and coord_best1[k-m] != dbdd.u[0, k]:
                print("count_both_not_good!", proba_best2[k], proba_best[k-m])
                count_both_not_good += 1
                print("Comparison (best2, best1, u): ", coord_best2[k], coord_best1[k-m], dbdd.u[0, k]) 
        else:
            if k - m in proba_best.keys():
                # print("k = ", k)
                # print("k-m = ", k-m)
                # print(proba_best2[k])
                # print(proba_best[k-m])
                proba_best2[k] = proba_best[k-m]
                # print("Comparison (best2=best1, u): ", coord_best1[k-m], dbdd.u[0, k]) 
                coord_best2[k] = coord_best1[k-m]
                count_total_guess12 += 1
                if coord_best1[k-m] != dbdd.u[0, k]:
                    print("count_both_not_good (12 same)!", proba_best[k-m])
                    print("Comparison (best2=best1, u): ", coord_best1[k-m], dbdd.u[0, k]) 
                    count_both_not_good += 1
            # else:
            #     proba_best2[k] = 0 #assign a minimal value for positions corresponding to e.
    print("count_guess2_better = ", count_guess2_better)
    print("count_guess1_better = ", count_guess1_better)
    print("count_total_guess12 = ", count_total_guess12)

    (beta_before_guess, _) = dbdd.estimate_attack()
    if to_print:
        print("perfect hint after intersect_beta: ", beta_before_guess)
    # dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(0, m)) #temp indices=range(n, n + m)
    # (beta, _) = dbdd.estimate_attack()
    # if to_print:
    #     print("perfect hint after intersect_beta after q: ", beta)
#####Guess phase#####
        # if k < m:
        #     print("prob_best for e", proba_best2[k])
        # if k >= m and k < n+m:
            
        #     if proba_best[k-m] > proba_best2[k]:
        #         count_bad += 1
                # print("badddd")
                # print("before/after intersection, prob_best", k, proba_best[k-n], proba_best2[k])
                # print("before/after intersection, var", k, list_var[k-n], intersect_var[k], intersect_av[k])
    print("bad count: ", count_bad)
    # sorted_guesses = sorted(proba_best2.items(),
    #                         key=lambda kv: - kv[1]) #sorted index ############################

    sorted_guesses1 = sorted(proba_best.items(),
                            key=lambda kv: - kv[1]) #sorted index

    sorted_guesses1 = [sorted_guesses1[i][0] for i in range(len(sorted_guesses1))]

    sorted_guesses2 = sorted(proba_best2.items(),
                            key=lambda kv: - kv[1]) #sorted index

    sorted_guesses2 = [sorted_guesses2[i][0] for i in range(len(sorted_guesses2))]

   
    proba_success = 1.
    dbdd.verbosity = 0
    guesses = 0
    report_every = None


    count_guess2_better_g = 0 ###############
    count_guess1_better_g = 0 ###############
    count_both_not_good_g = 0 ################
    set_guess2_better_g = {} ###############
    set_guess1_better_g = {} ###############
    set_both_not_good_g = {} ################
    last_included_prob = 0 ################
    set_guess_g = {}  ######################
    secret_vec = dbdd.u[0]
    for t in sorted_guesses2: #############
        # print("(guesses, max_guesses)=", guesses, max_guesses)
        if proba_success*proba_best2[t] >= 0.77:#proba_success*proba_best[t-m] >= 0.77:#proba_success*proba_best2[t] >= 0.77: #(guesses <= max_guesses):(guesses <= max_guesses): proba_success*proba_best2[t] >= max_guesses  #################################
            # print("comparing prob_best in guess phase: ", proba_best[t-n], proba_best2[t])
            v = vec(Id[t])
            if dbdd.integrate_perfect_hint(v, _, force = False):
                guesses += 1
                proba_success *= proba_best2[t] #proba_best2[t]  #####################################################
                last_included_prob = proba_best2[t] #proba_best2[t]  ############################################################
                set_guess_g[t] = proba_best2[t]
                if coord_best2[t] != dbdd.u[0, t] and coord_best1[t-m] == dbdd.u[0, t]:
                    print(guesses, "count_guess1_better_g!", t, t-m, proba_best2[t], proba_best[t-m])
                    count_guess1_better_g += 1
                    set_guess1_better_g[t] = proba_best2[t]
                if coord_best2[t] == dbdd.u[0, t] and coord_best1[t-m] != dbdd.u[0, t]:
                    print(guesses, "count_guess2_better_g!", t, t-m, proba_best2[t], proba_best[t-m])
                    count_guess2_better_g += 1
                    set_guess2_better_g[t] = proba_best2[t]
                if coord_best2[t] != dbdd.u[0, t] and coord_best1[t-m] != dbdd.u[0, t]:
                    print(guesses, "count_both_not_good_g!", t, t-m, proba_best2[t], proba_best[t-m])
                    count_both_not_good_g += 1
                    set_both_not_good_g[t] = proba_best2[t]
                
            # if report_every is not None and (j % report_every == 0):
            #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(0, m))
    print("-----real guess----")
    print("count_guess2_better_g = ", count_guess2_better_g)
    print("count_guess1_better_g = ", count_guess1_better_g)
    print("count_both_not_good_g = ", count_both_not_good_g)
    print("last guess probability included", last_included_prob)

    if not (len(set_guess1_better_g) == 0 and len(set_guess2_better_g) == 0 and len(set_both_not_good_g) == 0):
        proba_success1 = 1.
        guesses1 = 0
        count_guess2_better_1 = 0 ###############
        count_guess1_better_1 = 0 ###############
        count_both_not_good_1 = 0 ################
        set_guess2_better_1 = {} ###############
        set_guess1_better_1 = {} ###############
        set_both_not_good_1 = {} ################
        last_included_prob1 = 0 ################
        set_guess_1 = {} ##################################
        for t_semi in sorted_guesses1: #############
            t = t_semi + m ##############################
            # print("(guesses, max_guesses)=", guesses, max_guesses)
            if proba_success1*proba_best[t-m] >= 0.77:#proba_success*proba_best2[t] >= 0.77: #(guesses <= max_guesses):(guesses <= max_guesses): proba_success*proba_best2[t] >= max_guesses  #################################
                # print("comparing prob_best in guess phase: ", proba_best[t-n], proba_best2[t])
                v = vec(Id[t])
                #perfect hint deleted
                guesses1 += 1
                proba_success1 *= proba_best[t-m] #proba_best2[t]  #####################################################
                last_included_prob1 = proba_best[t-m] #proba_best2[t]  ############################################################
                set_guess_1[t] = proba_best[t-m]
                if coord_best2[t] != secret_vec[t] and coord_best1[t-m] == secret_vec[t]:
                    print(guesses1, "count_guess1_better_1!", t, t-m, proba_best2[t], proba_best[t-m])
                    count_guess1_better_1 += 1
                    set_guess1_better_1[t] = proba_best[t-m]
                if coord_best2[t] == secret_vec[t] and coord_best1[t-m] !=secret_vec[t]:
                    print(guesses1, "count_guess2_better_1!", t, t-m, proba_best2[t], proba_best[t-m])
                    count_guess2_better_1 += 1
                    set_guess2_better_1[t] = proba_best[t-m]
                if coord_best2[t] != secret_vec[t] and coord_best1[t-m] != secret_vec[t]:
                    print(guesses1, "count_both_not_good_1!", t, t-m, proba_best2[t], proba_best[t-m])
                    count_both_not_good_1 += 1
                    set_both_not_good_1[t] = proba_best[t-m]

        print("-----Testing guess_1----")
        print("count_guess2_better_1 = ", count_guess2_better_1)
        print("count_guess1_better_1 = ", count_guess1_better_1)
        print("count_both_not_good_1 = ", count_both_not_good_1)
        print("# of guesses, last guess probability included", guesses1, last_included_prob1)
        testing_data2 = [set_guess_0, set_guess2_better_g, set_guess1_better_g, set_both_not_good_g, set_guess_g, set_guess2_better_1, set_guess1_better_1, set_both_not_good_1, set_guess_1]
    else:
        testing_data2 = []

    beta_before_last_q = dbdd.beta
    print("beta_before_last_q", beta_before_last_q)
    if report_every is not None:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(0, m), report_every=report_every)
    else:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(0, m))
    print("beta after guess and q", dbdd.beta)
    testing_data = [proba_best, proba_best2, coord_best1, coord_best2]
    return beta_before_guess, dbdd.beta, proba_success, guesses, beta_before_last_q, testing_data, testing_data2 #, dict1, dict2


In [12]:
# def reconstruct_diagonal_S(list_av, list_var, D_s, D_e, m, n, intersect_indices, to_print):
##Selected indices
    
#     mu_e, s_e = average_variance(D_e)
#     mu_s, s_s = average_variance(D_s)
    
#     #ellipsoids of first n dimension, rescaling to <= 1
#     Sigma_SCA = diagonal_matrix(list_var)*n
#     mu_SCA = vec(list_av)
#     Sigma_LWE = matrix(Sigma_SCA)*n
#     mu_LWE = matrix(mu_SCA)
   
#     for i in intersect_indices:
#         Sigma_LWE[i, i] = s_s*n
#         mu_LWE[0, i] = mu_s
        
      
    
#     logging("order B: input1 SCA ellip, input2 LWE ellip", style="HEADER")
#     ellipsoid_intersection(mu_SCA , Sigma_SCA, mu_LWE, Sigma_LWE, tolerance=1.48e-08)
#     logging("order A: input1 LWE ellip, intput2 SCA ellip", style="HEADER")
#     new_mu_sub, new_S_sub = ellipsoid_intersection(mu_LWE, Sigma_LWE, mu_SCA , Sigma_SCA, tolerance=1.48e-08)
    
#     new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub)
#     new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1))
#     rest_S = diagonal_matrix(m * [s_e] + n * [0] + [1])
    
#     hybrid_S = (new_S/n + rest_S)
#     diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])
    
#     new_mu = concatenate([new_mu_sub, [mu_e]*m + [0]])
#     return diagonal_S, new_mu
def reconstruct_diagonal_S_nonsingular(list_av, list_var, D_s, D_e, m, n, intersect_indices, to_print):
    ##full n submatrix
    full = False
    add_poster = True
    if full:
        mu_SCA = vec(list_av)
        S_SCA = diagonal_matrix(list_var)*(n)
        
        mu_e, s_e = average_variance(D_e)
        mu_s, s_s = average_variance(D_s)
        mu_LWE = vec(n * [mu_s])
        S_LWE = diagonal_matrix(n * [s_s])*(n)
    
    else:

        ####partial coordinates###
        mu_s, s_s = average_variance(D_s)
        mu_e, s_e = average_variance(D_e)
        
        #ellipsoids of first n dimension, rescaling to <= 1
        S_SCA = diagonal_matrix(list_var)*n
        # if to_print:
            # print("1. list_var", list_var)
            # print("1. ln det(SCA matrix): ", logdet(S_SCA).n())#(S_SCA.det()).n(), ln(S_SCA.det()).n())
        mu_SCA = vec(list_av)
        S_LWE = matrix(S_SCA)
        mu_LWE = matrix(mu_SCA)

        for i in intersect_indices:
            # if to_print:
            #     print("ebdd.S should be close to Sigma_SCA")
            #     print("s_s, ebdd.S (rescaling to <= 1", s_s, ebdd.S[i,i]*n)
            #     assert(ebdd.S[i,i]*n - Sigma_SCA[i,i] < 10**-5)
            S_LWE[i, i] = s_s*n #<=1
            mu_LWE[0, i] = mu_s
        ##########################
        if to_print:
            print("1. ln det(LWE matrix): ", logdet(S_LWE))#(S_LWE.det()).n(), ln(S_LWE.det()).n())
            # print("1. ln det(SCA matrix) - ln det(LWE matrix)", logdet(S_SCA) - logdet(S_LWE))#(S_SCA.det()).n() / (S_LWE.det()).n(), ln(S_SCA.det()).n() - ln(S_LWE.det()).n())

    if to_print:
        partial_u = vec(dbdd.u[0, n:n+m])

        #S_SCA_full:
        S_SCA_SE = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), S_SCA) #<=1
        S_SCA_SE1 = block4(S_SCA_SE, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
        rest_S_SCA = diagonal_matrix(m * [s_e] + n * [0] + [1]) #<=m
    
        S_SCA_full = (S_SCA_SE1/n + rest_S_SCA) #<=n+m+1
        #mu_SCA_full
        mu_SCA_full = concatenate([[mu_e]*m, mu_SCA, [0]])

        #S_LWE_full:
        S_LWE_SE = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), S_LWE) #<=1
        S_LWE_SE1 = block4(S_LWE_SE, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
        rest_S_LWE = diagonal_matrix(m * [s_e] + n * [0] + [1]) #<=m
    
        S_LWE_full = (S_LWE_SE1/n + rest_S_LWE) #<=n+m+1
        #mu_LWE_full
        mu_LWE_full = concatenate([[mu_e]*m, mu_LWE, [0]])
        # print("partial ellip_norm of input1 SCA ellip (dim = n): ")
        # print(scal((partial_u - mu_SCA) * matrix(np_inv(S_SCA)) * (partial_u - mu_SCA).T)) #should be <=1

        # print("full ellip_norm of input1 SCA ellip (dim = n + m + 1, should be <=1): ")
        # print(scal((dbdd.u - mu_SCA_full) * matrix(np_inv(S_SCA_full))/(n + m + 1) * (dbdd.u - mu_SCA_full).T)) #should be <=1

        print("partial ellip_norm of input1 LWE ellip (dim = n): ")
        print(scal((partial_u - mu_LWE) * matrix(np_inv(S_LWE)) * (partial_u - mu_LWE).T)) #should be <=1
        print("full ellip_norm of input1 LWE ellip  (dim = n + m + 1, should be <=1): ")
        print(scal((dbdd.u - mu_LWE_full) * matrix(np_inv(S_LWE_full))/(n + m + 1) * (dbdd.u - mu_LWE_full).T)) #should be <=1
    if to_print:
        print("ln det(SCA matrix): ", logdet(S_SCA))#(S_SCA.det()).n(), ln(S_SCA.det()).n())
        print("ln det(LWE matrix): ", logdet(S_LWE))#(S_LWE.det()).n(), ln(S_LWE.det()).n())
        # print("ln det(SCA matrix) - ln det(LWE matrix)", logdet(S_SCA) - logdet(S_LWE))#(S_SCA.det()).n() / (S_LWE.det()).n(), ln(S_SCA.det()).n() - ln(S_LWE.det()).n())
    # logging("order B: input1 SCA ellip, input2 LWE ellip", style="HEADER")
    # ellipsoid_intersection(mu_SCA, S_SCA, mu_LWE, S_LWE, tolerance=1.48e-08)
    # logging("order A: input1 LWE ellip, intput2 SCA ellip", style="HEADER")
    
    if to_print:
        import time
        start = time.time()
    new_mu_sub, new_S_sub = ellipsoid_intersection(mu_LWE, S_LWE, mu_SCA, S_SCA, tolerance=1.48e-08) #<=1
    if to_print:
        end = time.time()
        print("total time for ellipsoid_intersection:", end - start)
    # if to_print:
    #     print("matrix after intersection: ", new_S_sub[:5,:5])
    if to_print:
        print("ln det(intersected matrix): ",  logdet(new_S_sub))
    
    new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub) #<=1
    new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
    rest_S = diagonal_matrix(m * [s_e] + n * [0] + [1]) #<=m
    
    hybrid_S = (new_S/n + rest_S) #<=n+m+1
    diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])
    # if to_print:
    #     print(diagonal_S[n:n+5])
    if add_poster:
        diagonal_SCA = np.array([RR(S_SCA_full[i, i]) for i in range(hybrid_S.nrows())])
    new_mu = concatenate([[mu_e]*m, new_mu_sub, [0]])
    return diagonal_S, new_mu, diagonal_SCA, mu_SCA_full
def var_to_prob(var, mean, D_s):
    discrete_prob_list = [gaussian_prob(var, mean, x) for x in D_s.keys()]
    # print(D_s.keys())
    # print(discrete_prob_list)
    norm = sum(discrete_prob_list)
    # print(max(discrete_prob_list), norm)
    # denom = sqrt(var * 2 * pi)
    # ex = exp(-0.5 * (x - mean)**2 / var)
    # print("ex/denom", ex/denom)
    # print("gaussian_prob", gaussian_prob(var, mean, x), norm)
    ###To compare probability
    prob_tuple_list = [(x, gaussian_prob(var, mean, x)) for x in D_s.keys()]
    
    coord_best2 = max(prob_tuple_list, key=lambda x:x[1])[0]
    ###
    return max(discrete_prob_list)/norm, coord_best2
def gaussian_prob(var, mean, x):
    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
    # print("in gaussian_prob: ", var, mean, x, denom, ex)
    return 1/denom * ex

In [53]:
def reconstruct_diagonal_S(list_av, list_var, D_s, D_e, m, n, intersect_indices):###

    add_poster = True
    to_print = False
    #in case var = 0.0
    list_var_varnonzero = []
    list_av_varnonzero = []
    map_nonzero_to_full = {}
    map_full_to_nonzero = {}
    index_varnonzero = 0
    for index in range(len(list_var)):
        map_full_to_nonzero[index] = index_varnonzero
        if list_var[index] != 0.0:
            list_var_varnonzero += [list_var[index]]
            list_av_varnonzero += [list_av[index]]
            map_nonzero_to_full[index_varnonzero] = index
            index_varnonzero += 1


    intersect_indices_varnonzero = [map_full_to_nonzero[i] for i in intersect_indices]

 

    ####partial coordinates###
    mu_s, s_s = average_variance(D_s)
    mu_e, s_e = average_variance(D_e)
    
    #testing
    print("size of s with nonzero var v.s. n", len(list_var_varnonzero), n)

    n_varnonzero = len(list_var_varnonzero)
    #ellipsoids of first n_varnonzero dimension, rescaling to <= 1
    S_SCA = diagonal_matrix(list_var_varnonzero)*n_varnonzero

    mu_SCA = vec(list_av_varnonzero)
    S_LWE = matrix(S_SCA)
    mu_LWE = matrix(mu_SCA)

    for i in intersect_indices_varnonzero:

        S_LWE[i, i] = s_s*n_varnonzero #<=1
        mu_LWE[0, i] = mu_s
    ##########################
    if to_print:
        print("1. ln det(LWE matrix): ", logdet(S_LWE))
   
    if to_print:
        print("ln det(SCA matrix): ", logdet(S_SCA))#(S_SCA.det()).n(), ln(S_SCA.det()).n())
        print("ln det(LWE matrix): ", logdet(S_LWE))#(S_LWE.det()).n(), ln(S_LWE.det()).n())

    

    new_mu_sub_varnonzero, new_S_sub_varnonzero = ellipsoid_intersection(mu_LWE, S_LWE, mu_SCA, S_SCA, tolerance=1.48e-08) #<=1
 

    if to_print:
        print("ln det(intersected matrix): ",  logdet(new_S_sub_varnonzero))

    #add back the indices with var = 0.0
    new_S_sub_varnonzero_arr = np.array([RR(new_S_sub_varnonzero[i, i]) for i in range(new_S_sub_varnonzero.nrows())])
    new_S_sub_varnonzero_list = new_S_sub_varnonzero_arr.tolist()
    new_mu_sub_varnonzero_list = list(new_mu_sub_varnonzero[0])
    new_S_sub_list = [0]*n
    new_mu_sub_list = list(list_av)
    for index_varnonzero in range(n_varnonzero):
        index_full = map_nonzero_to_full[index_varnonzero]
        new_S_sub_list[index_full] = new_S_sub_varnonzero_list[index_varnonzero]
        new_mu_sub_list[index_full] = new_mu_sub_varnonzero_list[index_varnonzero]

    new_S_sub = diagonal_matrix(new_S_sub_list)
    new_mu_sub = vec(new_mu_sub_list)
 
    new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub) #<=1
    new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
    rest_S = diagonal_matrix(m * [s_e] + n * [0] + [0]) #<=m
    
    hybrid_S = (new_S/n + rest_S) #<=n+m+1
    diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])

    if add_poster:
        S_SCA_poster = diagonal_matrix(list_var)*n
        mu_SCA_poster = vec(list_av)
        S_SCA_SE_poster = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), S_SCA_poster) #<=1
        S_SCA_SE1_poster = block4(S_SCA_SE_poster, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
        rest_S_SCA_poster = diagonal_matrix(m * [s_e] + n * [0] + [0]) #<=m
    
        S_SCA_full_poster = (S_SCA_SE1_poster/n + rest_S_SCA_poster) #<=n+m+1
        #mu_SCA_full
        mu_SCA_full_poster = concatenate([[mu_e]*m, mu_SCA_poster, [0]])
        
        diagonal_SCA_poster = np.array([RR(S_SCA_full_poster[i, i]) for i in range(hybrid_S.nrows())])
    new_mu = concatenate([[mu_e]*m, new_mu_sub, [1]])
    return diagonal_S, new_mu, diagonal_SCA_poster, mu_SCA_full_poster


In [11]:
def data_to_measure(sca_seeds, params, param, frodo_distribution): 
    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]
    return measures
def simu_measured(secret, measures):
    """
    This fonction simulates the information gained by
    Bos et al attack. For a given secret, the simulation
    is outputting a score table at random from the data of Bos et al.
    :secret: an integer being the secret value
    :measurement: a score table
    """
    if secret in measures.keys():
        measurement = random.choice(measures[secret])
    else: 
        measurement = None
    return measurement

def measurement_to_aposteriori(measurement):
    """
    This fonction transforms a score table into an aposteriori distribution.
    According to the matlab code of Bos et al. attack,
    the score table is proportional
    to the logarithm of the aposteriori probability. We thus apply the exponential
    and renormalize.
    :measurement: a score table
    :apost_dist: a dictionnary of the aposteriori distribution
    """
    if measurement is not None:
        s = sum([exp(meas) for meas in measurement])
        measurement = [exp(meas)/s for meas in measurement]
        apost_dist = {}
        for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
            apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
    else:
        apost_dist = None
    return apost_dist


In [4]:
vec_a = vec({1,2,3})

In [45]:
list_var = [1, 0, 0, 1, 3,0, 2,5]
list_av = [0, 1,2,3,4,5,6, 7]

In [46]:
list_var_varnonzero = []
list_av_varnonzero = []
map_nonzero_to_full = {}
map_full_to_nonzero = {}
index_varnonzero = 0
for index in range(len(list_var)):
    map_full_to_nonzero[index] = index_varnonzero
    if list_var[index] != 0.0:
        list_var_varnonzero += [list_var[index]]
        list_av_varnonzero += [list_av[index]]
        map_nonzero_to_full[index_varnonzero] = index
        index_varnonzero += 1

In [48]:
intersect_indices = [0,3,4]

In [52]:
intersect_indices_varnonzero = [map_full_to_nonzero[i] for i in intersect_indices]
intersect_indices_varnonzero

[0, 1, 2]

In [50]:
def reconstruct_diagonal_S(list_av, list_var, D_s, D_e, m, n, intersect_indices):###

    add_poster = True
    to_print = False
    #in case var = 0.0
    list_var_varnonzero = []
    list_av_varnonzero = []
    map_nonzero_to_full = {}
    map_full_to_nonzero = {}
    index_varnonzero = 0
    for index in range(len(list_var)):
        map_full_to_nonzero[index] = index_varnonzero
        if list_var[index] != 0.0:
            list_var_varnonzero += [list_var[index]]
            list_av_varnonzero += [list_av[index]]
            map_nonzero_to_full[index_varnonzero] = index
            index_varnonzero += 1
    print("map_nonzero_to_full", map_nonzero_to_full)
    print("intersect_indices",intersect_indices)
    print("map_full_to_nonzero", map_full_to_nonzero)
    for i in intersect_indices:
        print(i)
        print(map_full_to_nonzero[i])
    intersect_indices_varnonzero = [map_full_to_nonzero[i] for i in intersect_indices]

    #testing:
    for i in intersect_indices:
        if list_var[i] == 0.0:
            print("intersect_indices should not contain indices with var = 0.0")

    ####partial coordinates###
    mu_s, s_s = average_variance(D_s)
    mu_e, s_e = average_variance(D_e)
    
    #testing
    print("size of s with nonzero var v.s. n", len(list_var_varnonzero), n)

    n_varnonzero = len(list_var_varnonzero)
    #ellipsoids of first n_varnonzero dimension, rescaling to <= 1
    S_SCA = diagonal_matrix(list_var_varnonzero)*n_varnonzero

    mu_SCA = vec(list_av_varnonzero)
    S_LWE = matrix(S_SCA)
    mu_LWE = matrix(mu_SCA)

    for i in intersect_indices_varnonzero:
        # if to_print:
        #     print("ebdd.S should be close to Sigma_SCA")
        #     print("s_s, ebdd.S (rescaling to <= 1", s_s, ebdd.S[i,i]*n)
        #     assert(ebdd.S[i,i]*n - Sigma_SCA[i,i] < 10**-5)
        S_LWE[i, i] = s_s*n_varnonzero #<=1
        mu_LWE[0, i] = mu_s
    ##########################
    if to_print:
        print("1. ln det(LWE matrix): ", logdet(S_LWE))#(S_LWE.det()).n(), ln(S_LWE.det()).n())
        # print("1. ln det(SCA matrix) - ln det(LWE matrix)", logdet(S_SCA) - logdet(S_LWE))#(S_SCA.det()).n() / (S_LWE.det()).n(), ln(S_SCA.det()).n() - ln(S_LWE.det()).n())

   
    if to_print:
        print("ln det(SCA matrix): ", logdet(S_SCA))#(S_SCA.det()).n(), ln(S_SCA.det()).n())
        print("ln det(LWE matrix): ", logdet(S_LWE))#(S_LWE.det()).n(), ln(S_LWE.det()).n())
        # print("ln det(SCA matrix) - ln det(LWE matrix)", logdet(S_SCA) - logdet(S_LWE))#(S_SCA.det()).n() / (S_LWE.det()).n(), ln(S_SCA.det()).n() - ln(S_LWE.det()).n())
    # logging("order B: input1 SCA ellip, input2 LWE ellip", style="HEADER")
    # ellipsoid_intersection(mu_SCA, S_SCA, mu_LWE, S_LWE, tolerance=1.48e-08)
    # logging("order A: input1 LWE ellip, intput2 SCA ellip", style="HEADER")
    

    new_mu_sub_varnonzero, new_S_sub_varnonzero = ellipsoid_intersection(mu_LWE, S_LWE, mu_SCA, S_SCA, tolerance=1.48e-08) #<=1
    #testing
    if to_print:
        print("matrix after intersection: ", new_S_sub_varnonzero[:5,:5])
        print("should be a diagonal matrix")

    if to_print:
        print("ln det(intersected matrix): ",  logdet(new_S_sub_varnonzero))

    #add back the indices with var = 0.0
    new_S_sub_varnonzero_arr = np.array([RR(new_S_sub_varnonzero[i, i]) for i in range(new_S_sub_varnonzero.nrows())])
    new_S_sub_varnonzero_list = new_S_sub_varnonzero_arr.tolist()
    new_mu_sub_varnonzero_list = list(new_mu_sub_varnonzero[0])
    new_S_sub_list = [0]*n
    new_mu_sub_list = list(list_av)
    for index_varnonzero in range(n_varnonzero):
        index_full = map_nonzero_to_full[index_varnonzero]
        new_S_sub_list[index_full] = new_S_sub_varnonzero_list[index_varnonzero]
        new_mu_sub_list[index_full] = new_mu_sub_varnonzero_list[index_varnonzero]

    new_S_sub = diagonal_matrix(new_S_sub_list)
    new_mu_sub = vec(new_mu_sub_list)
 
    new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub) #<=1
    new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
    rest_S = diagonal_matrix(m * [s_e] + n * [0] + [0]) #<=m
    
    hybrid_S = (new_S/n + rest_S) #<=n+m+1
    diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])
    # if to_print:
    #     print(diagonal_S[n:n+5])
    if add_poster:
        S_SCA_poster = diagonal_matrix(list_var)*n
        mu_SCA_poster = vec(list_av)
        S_SCA_SE_poster = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), S_SCA_poster) #<=1
        S_SCA_SE1_poster = block4(S_SCA_SE_poster, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
        rest_S_SCA_poster = diagonal_matrix(m * [s_e] + n * [0] + [0]) #<=m
    
        S_SCA_full_poster = (S_SCA_SE1_poster/n + rest_S_SCA_poster) #<=n+m+1
        #mu_SCA_full
        mu_SCA_full_poster = concatenate([[mu_e]*m, mu_SCA_poster, [0]])
        
        diagonal_SCA_poster = np.array([RR(S_SCA_full_poster[i, i]) for i in range(hybrid_S.nrows())])
    new_mu = concatenate([[mu_e]*m, new_mu_sub, [1]])
    return diagonal_S, new_mu, diagonal_SCA_poster, mu_SCA_full_poster

    #to fix diagonal_SCA and mu_SCA_full in the big chunk of to_print condition.

{0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 3, 6: 3, 7: 4}